In [1]:
from dataclasses import dataclass
from typing import Union, Type, Any
import re

- Classes: Setting, Variable, Unit
- Methods of Variable: add, sub, multi, negative (magic variables)
- Methods of Unit: mult, inverse $\left (\frac{1}{Unit} \right )$

In [2]:
STANDARD_UNIT_PREFIXES = {
    'n': 1e-9,
    'u': 1e-6,
    'm': 1e-3,
    'c': 1e-2,
    'd': 1e-1,
    '': 1,
    'k': 1e3,
    'M': 1e6,
    'G': 1e9,
    'T': 1e12,
}

In [3]:
@dataclass 
class Unit:
    """
    Класс единиц измерений
    """
    m: int
    s: int
    kg: int
    Hz: int
    B: int
        
    def __mul__(self, other):
        """
        Операция перемножения степеней 
        """
        u = Unit
        u.m = self.m + other.m
        u.s = self.s + other.s
        u.kg = self.kg + other.kg
        u.Hz = self.Hz + other.Hz
        u.B = self.B + other.B
        
        return u
    
    def inverse(self):
        """
        Операция обращение степеней
        Пример: kg^3 s^2 -> kg^-3 s^-2
        """
        self.m = -self.m
        self.s = -self.s
        self.kg = -self.kg
        self.Hz = -self.Hz
        self.B = -self.B
            

In [5]:
@dataclass 
class Variable:
    """
    Класс переменной 
    """
    name: ''
    value: Union[int, float, bool, str]
    unit: Unit
    description: ''
        
    def __add__(self, other):
        """
        Сложение класса Variable
        
        Знак операции +
        """
        
        assert(self.unit.m == other.unit.m)
        assert(self.unit.s == other.unit.s)
        assert(self.unit.kg == other.unit.kg)
        assert(self.unit.Hz == other.unit.Hz)
        assert(self.unit.B == other.unit.B)
        
        var = Variable(name='',
                       value=self.value + other.value,
                       unit=self.unit,
                       description=f'Sum of {self.name} and {other.name}' )
        
        return var
    
    def __sub__(self, other):
        """
        Вычитание класса Variable
        
        Знак операции -
        """
        
        assert(self.unit.m == other.unit.m)
        assert(self.unit.s == other.unit.s)
        assert(self.unit.kg == other.unit.kg)
        assert(self.unit.Hz == other.unit.Hz)
        assert(self.unit.B == other.unit.B)
        
        var = Variable(name='',
                       value=self.value - other.value,
                       unit=self.unit,
                       description=f'Difference of {self.name} and {other.name}' )
        return var
    
    def __mul__(self, other):
        """
        Умножение класса Variable
        
        Знак операции *
        """
        
        u = self.unit * other.unit
        var = Variable(name='',
                       value=self.value + other.value,
                       unit=u,
                       description=f'The product of {self.name} and {other.name}')
        
        return var
    
    def __truediv__(self, other):
        """
        Деление класса Variable
        
        Знак операции /
        """
        
        other.unit.inverse()
        
        u = self.unit * other.unit
        
        var = Variable(name='',
                       value= self.value/other.value,
                       unit=u,
                       description=f'Fraction of {self.name} and {other.name}')
        
        return var
        
    def __neg__(self):
        
        self.value = -self.value

In [15]:
a = Unit(1,2,3 ,4,5)
b =  Unit(1,2,3 ,4,5)
c = Variable('ha', 3, a, 'smth')
d = Variable('ha', -3, b, 'smth')

In [175]:
a = c/d

In [185]:
@dataclass 
class Setting:
    """
    Класс настройки
    """
    name: ''
    default_value: Variable
    description: ''